# 🚕 Allocation en ligne de tâches
**Master 2 ANDROIDE - CoCoMA (2024-2025)**


## 🎯 Objectif du projet
Ce projet vise à simuler l'allocation décentralisée de tâches dans une flotte de taxis, en utilisant différentes approches comme l'optimisation de contraintes distribuées (DCOP) et les protocoles de négociation (enchères).

### 📝 Plan :
1. Génération de tâches en ligne
2. Optimisation de contraintes distribuées (DCOP)
3. Protocoles de négociation multi-agents


### 🧔🏻‍♂️👩🏻 Binôme :
- Joe CHAMOUN - 21312860
- Nour Ismahane SLIMANI - 21221230

# 📦 Importation des Bibliothèques

In [ ]:
import importlib.util
import os

def is_running_in_colab():
    return importlib.util.find_spec('google.colab') is not None

!pip install colorama

if is_running_in_colab():
    !pip uninstall -y pydcop
    !git clone https://github.com/Orange-OpenSource/pyDcop.git
    %cd pyDcop

    # Fixing import issues
    if is_running_in_colab():
        file_path = 'pydcop/dcop/yamldcop.py'

        with open(file_path, 'r') as file:
            lines = file.readlines()

        with open(file_path, 'w') as file:
            for line in lines:
                if 'from collections.abc import Iterable' in line:
                    file.write('from collections.abc import Iterable as CollectionIterable  # Fix for CollectionIterable\n')
                elif 'from collections import Iterable' in line:
                    file.write('from collections.abc import Iterable as CollectionIterable  # Fix for Python 3.10+\n')
                else:
                    file.write(line)

        print(f"Fixed 'CollectionIterable' import in {file_path}")

        ############################################################################

        pydcop_dir = 'pydcop'

        for root, dirs, files in os.walk(pydcop_dir):
            for file in files:
                if file.endswith('.py'):
                    file_path = os.path.join(root, file)
                    with open(file_path, 'r') as f:
                        content = f.read()

                    if 'from pulp.solvers import GLPK_CMD' in content:
                        content = content.replace('from pulp.solvers import GLPK_CMD', 'from pulp import GLPK_CMD  # Fixed import for PuLP')

                        with open(file_path, 'w') as f:
                            f.write(content)
                        print(f"Fixed import in: {file_path}")


    !pip install .
    %cd ..

!pip install pulp

%matplotlib inline

In [ ]:
import numpy as np
import random
import time
import copy
import pandas as pd
import subprocess
import json
import yaml
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from itertools import permutations, combinations, islice
from enum import Enum
from datetime import datetime
from IPython.display import HTML
from functools import lru_cache
from colorama import Fore, Style

colors_FORE = [
    Fore.RED, Fore.GREEN, Fore.YELLOW, Fore.BLUE,
    Fore.MAGENTA, Fore.CYAN, Fore.LIGHTRED_EX, Fore.LIGHTGREEN_EX,
    Fore.LIGHTBLUE_EX, Fore.LIGHTMAGENTA_EX, Fore.LIGHTCYAN_EX
]

colors_plot = [
    'red', 'green', 'yellow', 'blue',
    'magenta', 'cyan', 'lightcoral', 'lightgreen',
    'lightblue', 'violet', 'turquoise'
]


# Enum pour les types d'enchères
class AuctionType(Enum):
    NONE = 'none'
    PSI = 'PSI'  # Enchères parallèles
    SSI = 'SSI'  # Enchères séquentielles
    SSI_REGRET = 'SSI avec regret'  # Enchères séquentielles basées sur le regret


# 🏗️ Partie 1 : Génération de Tâches en Ligne

Dans cette section, nous allons :
1. Créer des classes pour représenter les taxis et les tâches.
2. Générer dynamiquement des trajets pour les taxis.
3. Simuler l'allocation des tâches et le déplacement des taxis.

## 1.1 Définition des Classes de Base
Création des classes `Task` pour représenter les trajets et `Taxi` pour les agents.

In [ ]:
# Classe pour représenter une tâche (trajet)
class Task:
    task_counter = 0  # Compteur de tâches pour attribuer des identifiants uniques

    def __init__(self, start, end):
        """
        Initialise une tâche avec un identifiant unique, une position de départ et une position d'arrivée.
        Le coût est calculé automatiquement comme la distance euclidienne entre les deux points.

        :param start: Tuple (x, y) indiquant la position de départ.
        :param end: Tuple (x, y) indiquant la position d'arrivée.
        """
        self.task_id = Task.task_counter
        Task.task_counter += 1
        self.start = start
        self.end = end
        self.cost = self.calculate_cost()  # Coût de la tâche calculé à partir de la distance
        self.is_assigned = False  # Statut de la tâche (assignée ou non)

    def calculate_cost(self):
        """
        Calcule le coût de la tâche en utilisant la distance euclidienne.

        :return: Distance entre la position de départ et d'arrivée.
        """
        return np.linalg.norm(np.array(self.end) - np.array(self.start))

    def __repr__(self):
        """
        Représentation textuelle de la tâche pour faciliter le débogage et l'affichage.
        """
        return f"Task(id={self.task_id}, start={self.start}, end={self.end}, is_assigned={self.is_assigned}, cost={self.cost:.2f})"


################################################################################


class Taxi:
    def __init__(self, taxi_id, position, environment):
        """
        Initialise un taxi avec un identifiant, une position initiale et une référence à l'environnement.

        :param taxi_id: Identifiant unique du taxi.
        :param position: Position initiale du taxi sous forme de tuple (x, y).
        :param environment: Référence à l'environnement dans lequel le taxi opère.
        """
        self.taxi_id = taxi_id
        self.position = position
        self.environment = environment  # Référence à l'environnement pour accéder aux tâches
        self.current_task = None  # Tâche en cours d'exécution
        self.execution_cost = 0  # Coût total d'exécution accumulé
        self.cost_over_time = []  # Stocke le coût à chaque étape
        self.task_phase = 'idle'  # Phases possibles: 'idle', 'moving_to_start', 'moving_to_end'
        self.schedule = []  # Planification optimisée des tâches assignées
        self.path = []  # Stocke les positions parcourues
        self.tasks_completed = 0

        # Assigner une couleur unique à chaque taxi pour l'affichage console (FORE)
        available_FORE_colors = [color for color in colors_FORE if color not in self.environment.used_colors_fore]
        if available_FORE_colors:
            self.color_fore = random.choice(available_FORE_colors)
        else:
            self.color_fore = random.choice(colors_FORE)
        self.environment.used_colors_fore.add(self.color_fore)

        # Obtenir l'index de la couleur FORE choisie
        color_index = colors_FORE.index(self.color_fore)

        # Assigner la même couleur pour matplotlib en utilisant l'index
        self.color_plot = colors_plot[color_index]
        self.environment.used_colors_plot.add(self.color_plot)

    def assign_task(self, task):
        """
        Assigne une tâche au taxi et l'ajoute au planning.

        :param task: Instance de la classe Task à assigner.
        """
        task.is_assigned = True
        self.schedule.append(task)

    @lru_cache(maxsize=None)
    def _calculate_schedule_cost(self, schedule_tuple):
        """
        Calcule le coût total d'un planning donné.

        :param schedule_tuple: Tuple des tâches dans le planning (utilisé pour le cache).
        :return: Coût total du planning.
        """
        schedule = list(schedule_tuple)  # Conversion du tuple en liste pour traitement
        total_cost = 0
        current_pos = self.position
        for task in schedule:
            total_cost += self.calculate_distance(current_pos, task.start)  # Coût pour atteindre le départ de la tâche
            total_cost += task.cost  # Coût d'exécution de la tâche
            current_pos = task.end  # Mise à jour de la position actuelle
        return total_cost

    def start_task(self):
        """
        Démarre la première tâche de la liste si aucune tâche n'est en cours.
        """
        if self.schedule and not self.current_task:
            self.current_task = self.schedule[0]
            self.task_phase = 'moving_to_start'
            self.path = [self.position]

    def update_position(self):
        """
        Met à jour la position du taxi en fonction de la tâche en cours.
        """
        self.cost_over_time.append(self.execution_cost)
        if self.current_task:
            if self.task_phase == 'moving_to_start':
                self.move_towards(self.current_task.start)
                if self.position == self.current_task.start:
                    self.task_phase = 'moving_to_end'

            elif self.task_phase == 'moving_to_end':
                self.move_towards(self.current_task.end)
                if self.position == self.current_task.end:
                    self.finish_task()
        else:
            self.start_task()

    def move_towards(self, target):
        """
        Déplace le taxi d'un pas vers la position cible.

        :param target: Position cible (x, y).
        """
        x, y = self.position
        tx, ty = target

        if x < tx: x += 1
        elif x > tx: x -= 1
        if y < ty: y += 1
        elif y > ty: y -= 1

        # Mettre à jour la position uniquement si elle a changé
        if (x, y) != self.position:
            self.position = (x, y)
            self.path.append(self.position)

    def finish_task(self):
        """
        Termine la tâche en cours, met à jour le coût et prépare le taxi pour la prochaine tâche.
        """
        if self.schedule:
            self.tasks_completed += 1  # Incrémenter à chaque tâche finie
            completed_task = self.schedule.pop(0)
            self.execution_cost += completed_task.cost
            if completed_task in self.environment.tasks:
                self.environment.tasks.remove(completed_task)  # Suppression de la tâche de l'environnement
            self.current_task = None
            self.task_phase = 'idle'
            self.path = []

    @lru_cache(maxsize=None)
    def calculate_distance(self, start, end):
        """
        Calcule la distance entre deux points.

        :param start: Position de départ (x, y).
        :param end: Position d'arrivée (x, y).
        :return: Distance euclidienne.
        """
        return np.linalg.norm(np.array(start) - np.array(end))

    def __repr__(self):
        """
        Représentation textuelle du taxi pour faciliter le débogage et l'affichage.
        """
        return f"Taxi(id={self.taxi_id}, total_cost={self.execution_cost:.2f}, position={self.position}, tasks={self.schedule})"


## 1.2 Création de l'Environnement et Simulation
Nous allons maintenant créer l'environnement qui simule le déplacement des taxis et l'arrivée des tâches.

In [ ]:
# Classe pour gérer l'environnement des taxis
class TaxiEnvironment:
    def __init__(self, grid_size, num_taxis, task_frequency, num_tasks=None, permutation_threshold=6, auction_type=AuctionType.NONE):
        """
        Initialise l'environnement avec une grille, des taxis, une fréquence de génération des tâches...

        :param grid_size: Taille de la grille.
        :param num_taxis: Nombre de taxis.
        :param task_frequency: Fréquence de génération des tâches.
        :param num_tasks: Nombre de tâches à générer.
        :param permutation_threshold: Seuil de permutation pour choisir la méthode d'assignation des tâches.
        :param auction_type: Type d'enchères à utiliser.
        """
        self.grid_size = grid_size  # Définir la taille de la grille
        self.num_taxis = num_taxis  # Nombre de taxis dans l'environnement
        self.task_frequency = task_frequency  # Fréquence de génération des tâches
        self.num_tasks = num_tasks  # Nombre de tâches à générer
        self.permutation_threshold = permutation_threshold  # Seuil de permutation pour l'assignation
        self.auction_type = auction_type  # Type d'enchères à utiliser

        self.used_colors_fore = set()  # Couleurs déjà utilisées pour l'affichage console
        self.used_colors_plot = set()  # Couleurs déjà utilisées pour l'affichage graphique
        self.taxis = [Taxi(i, self.random_position(), self) for i in range(num_taxis)]  # Initialiser les taxis
        self.tasks = []  # Liste des tâches en attente
        self.time_step = 0  # Compteur de pas de temps
        self.history = []  # Stocke les états pour l'animation

    def random_position(self):
        """
        Génère une position aléatoire dans la grille.

        :return: Tuple (x, y) représentant la position aléatoire.
        """
        return (random.randint(0, self.grid_size - 1), random.randint(0, self.grid_size - 1))

    def generate_task(self):
        """
        Génère une tâche avec des points de départ et d'arrivée distincts.

        :return: Instance de la classe Task.
        """
        start, end = (0, 0), (0, 0)
        while start == end:
            start = self.random_position()
            end = self.random_position()
        return Task(start, end)

    def calculate_marginal_cost(self, taxi, task):
        """
        Calcule le coût marginal d'assignation d'une tâche à un taxi.

        :param taxi: Instance du taxi concerné.
        :param task: Instance de la tâche à assigner.
        :return: Coût total de l'assignation.
        """
        if taxi.task_phase == 'idle':
            base_cost = np.linalg.norm(np.array(taxi.position) - np.array(task.start)) + task.cost
        else:
            remaining_distance = np.linalg.norm(np.array(taxi.position) - np.array(taxi.current_task.end))
            distance_to_new_task = np.linalg.norm(np.array(taxi.current_task.end) - np.array(task.start))
            base_cost = remaining_distance + distance_to_new_task + task.cost

        # surcharge_penalty = len(taxi.schedule) * 5  # Pénalité pour surcharge de tâches
        return round(base_cost, 2) # + surcharge_penalty, 2)

    def allocate_tasks(self):
        """
        Attribue les tâches aux taxis en utilisant une stratégie dynamique.

        Cette méthode combine deux approches :
        1. Assignation immédiate aux taxis inactifs :
        Si des taxis sont inactifs (sans tâche en cours), les tâches sont immédiatement
        attribuées à ces taxis en utilisant une approche gloutonne.

        2. Stratégie adaptative :
        - Si le nombre de tâches non assignées est faible (inférieur à un seuil défini, 'permutation_threshold'),
            une approche basée sur des permutations est utilisée pour optimiser l'assignation.
        - Si le nombre de tâches est élevé, une assignation gloutonne est préférée pour
            des raisons de performance.
        """
        # Identifier les tâches non assignées et les taxis inactifs
        unassigned_tasks = [task for task in self.tasks if not task.is_assigned]
        idle_taxis = [taxi for taxi in self.taxis if taxi.task_phase == 'idle']

        # Priorité à l'assignation des tâches aux taxis inactifs
        if idle_taxis:
            print(f"Attribution des tâches à {len(idle_taxis)} taxis inactifs.")
            self.allocate_tasks_greedy(taxis=idle_taxis, tasks=unassigned_tasks)
        else:
            # Basculer dynamiquement en fonction du nombre de tâches non assignées
            if len(unassigned_tasks) <= self.permutation_threshold:
                print("Utilisation de l'assignation par permutations.")
                self.allocate_tasks_with_permutations()
            else:
                print("Utilisation de l'assignation gloutonne.")
                self.allocate_tasks_greedy()

    def allocate_tasks_with_permutations(self):
        """
        Optimise l'assignation des tâches en explorant différentes permutations
        pour minimiser le coût total de l'exécution des taxis.

        Approche :
        - Toutes les combinaisons possibles des tâches sont générées jusqu'à un certain
          nombre de permutations pour limiter la complexité.
        - Chaque permutation est évaluée pour déterminer le coût global associé.
        - La permutation avec le coût total le plus faible est sélectionnée et appliquée.

        Caractéristiques :
        1. Optimisation Globale :
        Tente d'optimiser l'assignation sur l'ensemble des taxis actifs, prenant
        en compte les tâches en cours et planifiées.

        2. Limitation des Permutations :
        Le nombre de permutations est limité à un seuil pour éviter des calculs
        excessivement longs.
        """
        if not self.tasks:
            return

        # Tâches flexibles : nouvelles tâches + celles planifiées (hors tâche en cours)
        all_flexible_tasks = self.tasks + [
            task for taxi in self.taxis
            for task in taxi.schedule
            if task != taxi.current_task
        ]

        best_total_cost = float('inf')
        best_assignment = None

        # Limite de permutations
        max_permutations = 100
        limited_permutations = islice(permutations(all_flexible_tasks, len(all_flexible_tasks)), max_permutations)

        # Boucle sur les permutations limitées
        for perm in limited_permutations:
            current_assignment = {taxi: [] for taxi in self.taxis}
            total_cost = 0

            # Distribuer les tâches basées sur le coût marginal
            for task in perm:
                # Trouver le taxi pour lequel l'ajout de cette tâche coûte le moins cher
                best_taxi = None
                min_marginal_cost = float('inf')

                for taxi in self.taxis:
                    tentative_schedule = ([taxi.current_task] if taxi.current_task else []) + current_assignment[taxi] + [task]
                    marginal_cost = taxi._calculate_schedule_cost(tuple(tentative_schedule))

                    if marginal_cost < min_marginal_cost:
                        min_marginal_cost = marginal_cost
                        best_taxi = taxi

                current_assignment[best_taxi].append(task)
                total_cost += min_marginal_cost

                # Si le coût dépasse déjà le meilleur trouvé, abandonner cette permutation
                if total_cost >= best_total_cost:
                    break

            if total_cost < best_total_cost:
                best_total_cost = total_cost
                best_assignment = current_assignment

        # Appliquer la meilleure assignation trouvée
        if best_assignment:
            for taxi in self.taxis:
                taxi.schedule = ([taxi.current_task] if taxi.current_task else []) + best_assignment[taxi]

            # Marquer les tâches comme assignées
            for task in self.tasks:
                task.is_assigned = True

    def allocate_tasks_greedy(self, taxis=None, tasks=None):
        """
        Assigne les tâches une par une en minimisant le coût pour chaque tâche.
        Approche rapide mais moins optimale que les permutations.

        :param taxis: Liste des taxis à considérer pour l'assignation. Si None, tous les taxis seront considérés.
        :param tasks: Liste des tâches à assigner. Si None, toutes les tâches non assignées seront considérées.
        """
        if tasks is None:
            tasks = [task for task in self.tasks if not task.is_assigned]
        if taxis is None:
            taxis = self.taxis

        # Trier les tâches non assignées par coût décroissant pour traiter les tâches les plus coûteuses en premier
        unassigned_tasks = sorted([task for task in self.tasks if not task.is_assigned], key=lambda t: t.cost, reverse=True)

        for task in unassigned_tasks:
            best_taxi = None
            min_marginal_cost = float('inf')

            # Trouver le taxi pour lequel cette tâche ajoute le moins de coût
            for taxi in self.taxis:
                temp_schedule = ([taxi.current_task] if taxi.current_task else []) + taxi.schedule + [task]
                marginal_cost = taxi._calculate_schedule_cost(tuple(temp_schedule))

                if marginal_cost < min_marginal_cost:
                    min_marginal_cost = marginal_cost
                    best_taxi = taxi

            # Assigner la tâche au taxi le plus adapté
            if best_taxi:
                best_taxi.schedule.append(task)
                task.is_assigned = True
                print(f"Tâche {task.task_id} assignée au Taxi {best_taxi.taxi_id} avec un coût marginal de {min_marginal_cost:.2f}")

        print("--- Assignation des Tâches Terminée ---")

    def step(self):
        """
        Effectue une étape de simulation : génère des tâches et met à jour les positions des taxis.
        """
        self.time_step += 1

        if self.time_step % self.task_frequency == 0:
            num_taxis = len(self.taxis)
            num_new_tasks = np.random.randint(num_taxis, num_taxis + 3) if self.num_tasks is None else self.num_tasks
            new_tasks = [self.generate_task() for _ in range(num_new_tasks)]
            self.tasks.extend(new_tasks)

            if self.auction_type == AuctionType.PSI:
                self.run_parallel_auctions()
            elif self.auction_type == AuctionType.SSI:
                self.run_sequential_auctions(use_regret=False)
            elif self.auction_type == AuctionType.SSI_REGRET:
                self.run_sequential_auctions(use_regret=True)
            else:
                self.allocate_tasks()

        for taxi in self.taxis:
            taxi.update_position()

        self.history.append(self.capture_state())

    def capture_state(self):
        """
        Capture l'état actuel de l'environnement pour l'animation, incluant les taxis, leurs positions, et les tâches courantes.
        """
        state = {
            'taxis': [
                (taxi.taxi_id, copy.deepcopy(taxi.position),
                taxi.current_task.task_id if taxi.current_task else None,
                copy.deepcopy(taxi.path))
                for taxi in self.taxis
            ],
            'tasks': [
                (taxi.taxi_id, task.task_id, copy.deepcopy(task.start), copy.deepcopy(task.end))
                for taxi in self.taxis
                for task in taxi.schedule
            ]
        }
        return state

    def run_simulation(self, total_steps, delay=0.5, show_animation=True, save_to_csv=False):
        """
        Lance la simulation pour un nombre de pas de temps donné et affiche l'animation.

        :param total_steps: Nombre de pas de temps.
        :param delay: Délai entre les étapes pour l'animation.
        :param show_animation: Booléen pour afficher l'animation ou non.
        :param save_to_csv: Booléen pour sauvegarder les résultats dans des fichiers CSV.
        """
        Task.task_counter = 0

        for i in range(total_steps):
            self.step()
            if not show_animation:
                self.display_grid()
                time.sleep(delay)
        if show_animation:
            display(self.animate_simulation())
        self.log_performance(save_to_csv)


    ############################################################################
    ############################### VISUALIZATION ##############################
    ############################################################################

    def display_grid(self, verbose=False):
        """
        Affiche la grille de simulation dans la console avec la position des taxis et des tâches.

        - Chaque taxi est coloré selon sa couleur attribuée.
        - Les tâches en cours sont affichées en gras.
        - Les tâches non assignées sont affichées en noir.

        :param verbose: Si True, affiche des informations supplémentaires comme le statut des taxis.
        """
        # Initialisation de la grille vide
        grid = [[' . ' for _ in range(self.grid_size)] for _ in range(self.grid_size)]

        # Parcours de chaque taxi pour afficher leur position et leurs tâches
        for taxi in self.taxis:
            taxi_color = taxi.color_fore  # Couleur spécifique attribuée au taxi

            # Marquer la position actuelle du taxi
            x, y = taxi.position
            taxi_id_str = f'{taxi.taxi_id:02}'  # Formatage de l'ID du taxi sur 2 chiffres
            grid[y][x] = taxi_color + f'T{taxi_id_str}' + Style.RESET_ALL

            # Afficher les tâches assignées à ce taxi
            for task in taxi.schedule:
                sx, sy = task.start  # Position de départ de la tâche
                ex, ey = task.end    # Position d'arrivée de la tâche
                task_id_str = f'{task.task_id:02}'  # Formatage de l'ID de la tâche sur 2 chiffres

                # Affichage en gras pour la tâche en cours
                if task == taxi.current_task:
                    grid[sy][sx] = Style.BRIGHT + taxi_color + f'S{task_id_str}' + Style.RESET_ALL
                    grid[ey][ex] = Style.BRIGHT + taxi_color + f'E{task_id_str}' + Style.RESET_ALL
                else:
                    grid[sy][sx] = taxi_color + f'S{task_id_str}' + Style.RESET_ALL
                    grid[ey][ex] = taxi_color + f'E{task_id_str}' + Style.RESET_ALL

        # Affichage des tâches non assignées en noir
        for task in [t for t in self.tasks if not t.is_assigned]:
            sx, sy = task.start
            ex, ey = task.end
            task_id_str = f'{task.task_id:02}'
            grid[sy][sx] = Fore.BLACK + f'S{task_id_str}' + Style.RESET_ALL
            grid[ey][ex] = Fore.BLACK + f'E{task_id_str}' + Style.RESET_ALL

        # Affichage des informations supplémentaires si verbose est activé
        if verbose:
            print(f"Time Step: {self.time_step}")

        # Affichage de la grille
        for row in grid:
            print(' '.join(row))
        print('\n')

        # Affichage des statuts des taxis si verbose est activé
        if verbose:
            print("Statut des Taxis :")
            for taxi in self.taxis:
                print(taxi)
        print('\n')

    def animate_simulation(self):
        """
        Crée une animation de la simulation en utilisant Matplotlib.

        - Chaque taxi est coloré différemment.
        - Les tâches en cours sont mises en valeur avec des bordures noires.
        - Toutes les tâches (en cours et futures) sont affichées.
        - Les tâches non assignées sont affichées en noir.

        :return: HTML de l'animation pour affichage.
        """
        fig, ax = plt.subplots(figsize=(8, 8))
        ax.set_xlim(0, self.grid_size)
        ax.set_ylim(0, self.grid_size)
        ax.set_title('Simulation de la Flotte de Taxis avec Trajectoires')

        def update(frame):
            ax.clear()
            state = self.history[frame]

            for taxi_id, position, current_task_id, path in state['taxis']:
                taxi = next(t for t in self.taxis if t.taxi_id == taxi_id)
                taxi_color = taxi.color_plot

                # Affiche la position du taxi
                ax.scatter(position[0], position[1], c=taxi_color, s=100, label=f'Taxi {taxi_id}')

                # Affichage des tâches assignées à ce taxi
                for _, task_id, start, end in [t for t in state['tasks'] if t[0] == taxi_id]:
                    # Mettre en évidence la tâche en cours avec une bordure noire
                    if current_task_id == task_id:
                        ax.scatter(start[0], start[1], edgecolors='black', linewidths=2, marker='s', s=100, c=taxi_color)
                        ax.scatter(end[0], end[1], edgecolors='black', linewidths=2, marker='^', s=100, c=taxi_color)
                        if path:
                            path_x, path_y = zip(*path)
                            ax.plot(path_x, path_y, linestyle='--', color=taxi_color, alpha=0.7)
                    else:
                        ax.scatter(start[0], start[1], marker='s', s=100, c=taxi_color)
                        ax.scatter(end[0], end[1], marker='^', s=100, c=taxi_color)

            # Affiche les tâches non assignées en noir
            for task in [t for t in self.tasks if not t.is_assigned]:
                ax.scatter(task.start[0], task.start[1], c='black', marker='s', s=100)
                ax.scatter(task.end[0], task.end[1], c='black', marker='x', s=100)

            ax.set_xlim(0, self.grid_size)
            ax.set_ylim(0, self.grid_size)
            ax.set_title(f'Simulation de la Flotte de Taxis - Étape {frame}')
            ax.legend(loc='upper right')

        ani = animation.FuncAnimation(fig, update, frames=len(self.history), interval=500, blit=False)
        return HTML(ani.to_jshtml())


    ############################################################################
    ############################## LOGS and PLOTS ##############################
    ############################################################################

    def log_performance(self, save_to_csv=False):
        """
        Affiche les métriques de performance à la fin de la simulation et sauvegarde les résultats.
        """
        total_tasks = sum(len(taxi.schedule) for taxi in self.taxis)
        completed_tasks = Task.task_counter - total_tasks
        total_distance = sum(taxi.execution_cost for taxi in self.taxis)

        print("\n--- Résumé des Performances ---")
        print(f"Tâches complétées : {completed_tasks}")
        print(f"Distance totale parcourue par les taxis : {total_distance:.2f}")
        average_distance = total_distance / completed_tasks if completed_tasks > 0 else 0
        print(f"Distance moyenne par tâche : {average_distance:.2f}")

        self.plot_cost_evolution()
        self.plot_total_cost_per_taxi()
        self.plot_task_distribution()

        if save_to_csv:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            self.save_performance_to_csv(completed_tasks, total_distance, average_distance, timestamp)
            self.save_cost_evolution_to_csv(timestamp)

    def plot_cost_evolution(self):
        """
        Affiche l'évolution du coût pour chaque taxi au fil du temps.
        """
        plt.figure(figsize=(10, 6))
        for taxi in self.taxis:
            plt.plot(taxi.cost_over_time, label=f'Taxi {taxi.taxi_id}', linewidth=2, color=taxi.color_plot)

        plt.xlabel('Étapes de Simulation')
        plt.ylabel('Coût Cumulé')
        plt.title('Évolution du Coût par Taxi')
        plt.legend()
        plt.grid(True)
        plt.show()

    def plot_total_cost_per_taxi(self):
        """
        Affiche le coût total accumulé par chaque taxi.
        """
        total_costs = [taxi.execution_cost for taxi in self.taxis]
        taxi_ids = [f'Taxi {taxi.taxi_id}' for taxi in self.taxis]
        colors = [taxi.color_plot for taxi in self.taxis]

        plt.figure(figsize=(8, 5))
        plt.bar(taxi_ids, total_costs, color=colors)
        plt.xlabel('Taxis')
        plt.ylabel('Coût Total Accumulé')
        plt.title('Coût Total par Taxi')
        plt.show()

    def plot_task_distribution(self):
        """
        Affiche la distribution des tâches entre les taxis.
        """
        task_counts = [len(taxi.schedule) for taxi in self.taxis]
        taxi_ids = [f'Taxi {taxi.taxi_id}' for taxi in self.taxis]
        colors = [taxi.color_plot for taxi in self.taxis]

        plt.figure(figsize=(8, 6))
        plt.bar(taxi_ids, task_counts, color=colors)
        plt.xlabel('Taxis')
        plt.ylabel('Nombre de Tâches')
        plt.title('Distribution des Tâches entre les Taxis')
        plt.show()

    def plot_auction_cost_comparison(self, auction_costs):
        """
        Compare les coûts totaux entre les différents types d'enchères.

        :param auction_costs: Dictionnaire avec les coûts totaux par type d'enchère.
        """
        auction_types = [auction_type for auction_type in auction_costs.keys()]
        total_costs = list(auction_costs.values())

        plt.figure(figsize=(8, 6))
        plt.bar(auction_types, total_costs, color='red')
        plt.xlabel("Types d'Enchères")
        plt.ylabel('Coût Total')
        plt.title("Comparaison des Coûts Totaux entre les Différents Types d'Enchères")
        plt.show()

    def plot_completed_tasks_by_auction(self, completed_tasks):
        """
        Affiche le nombre de tâches complétées par type d'enchère.

        :param completed_tasks: Dictionnaire avec le nombre de tâches complétées par type d'enchère.
        """
        auction_types = [auction_type for auction_type in completed_tasks.keys()]
        task_counts = list(completed_tasks.values())

        plt.figure(figsize=(8, 6))
        plt.bar(auction_types, task_counts, color='lightgreen')
        plt.xlabel("Types d'Enchères")
        plt.ylabel('Tâches Complétées')
        plt.title("Nombre de Tâches Complétées par Type d'Enchère")
        plt.show()

    def plot_global_comparison(self, methods_performance):
        """
        Compare globalement les performances des méthodes (DCOP vs Enchères vs Gloutonnes).

        :param methods_performance: Dictionnaire avec les performances des différentes méthodes.
        """
        methods = list(methods_performance.keys())
        total_costs = list(methods_performance.values())

        plt.figure(figsize=(8, 6))
        plt.bar(methods, total_costs, color='teal')
        plt.xlabel('Méthodes')
        plt.ylabel('Coût Total')
        plt.title('Comparaison Globale des Méthodes')
        plt.show()

    def save_performance_to_csv(self, completed_tasks, total_distance, average_distance, timestamp):
        """
        Sauvegarde les métriques de performance dans un fichier CSV.

        :param completed_tasks: Nombre total de tâches complétées.
        :param total_distance: Distance totale parcourue par tous les taxis.
        :param average_distance: Distance moyenne par tâche.
        """
        data = {
            'ID Taxi': [taxi.taxi_id for taxi in self.taxis],
            'Distance Totale': total_distance,
            'Tâches Complétées': completed_tasks,
            'Distance Moyenne par Tâche': average_distance
        }

        df = pd.DataFrame(data)
        df.to_csv(f'resultats_performance_{timestamp}.csv', index=False)
        print(f"Les métriques de performance ont été sauvegardées dans 'resultats_performance_{timestamp}.csv'.")

    def save_cost_evolution_to_csv(self, timestamp):
        """
        Sauvegarde l'évolution des coûts de chaque taxi au fil des étapes dans un fichier CSV.
        """
        evolution_data = {
            'Étape': list(range(len(self.taxis[0].cost_over_time)))
        }

        for taxi in self.taxis:
            evolution_data[f'Taxi {taxi.taxi_id}'] = taxi.cost_over_time

        df = pd.DataFrame(evolution_data)
        df.to_csv(f'evolution_couts_{timestamp}.csv', index=False)
        print(f"L'évolution des coûts a été sauvegardée dans 'evolution_couts_{timestamp}.csv'.")


    ############################################################################
    ################################# AUCTIONS #################################
    ############################################################################

    def run_parallel_auctions(self):
        """
        Exécute des enchères parallèles pour assigner les tâches aux taxis.
        Chaque taxi soumet des offres en fonction du coût calculé.
        """
        print("\n--- Début des Enchères Parallèles ---")
        unassigned_tasks = [task for task in self.tasks if not task.is_assigned]

        if not unassigned_tasks:
            print("Aucune tâche à assigner.")
            return

        # 1ère étape: Chaque taxi fait des offres sur les tâches
        bids = {task.task_id: [] for task in unassigned_tasks}
        for taxi in self.taxis:
            for task in unassigned_tasks:
                cost = self.calculate_marginal_cost(taxi, task)
                bids[task.task_id].append((taxi, cost))
                print(f"Taxi {taxi.taxi_id} propose un coût de {cost:.2f} pour la tâche {task.task_id}.")

        # 2ème etape: Attribution des tâches à l'offre la plus basse
        for task in unassigned_tasks:
            if bids[task.task_id]:
                # Trier les offres par coût croissant
                bids[task.task_id].sort(key=lambda x: x[1])
                winning_taxi, winning_cost = bids[task.task_id][0]
                winning_taxi.assign_task(task)
                task.is_assigned = True
                print(f"Tâche {task.task_id} assignée au Taxi {winning_taxi.taxi_id} avec un coût de {winning_cost:.2f}.")
            else:
                print(f"Aucune offre reçue pour la tâche {task.task_id}.")

        print("--- Fin des Enchères Parallèles ---\n")

    def run_sequential_auctions(self, use_regret=False):
        """
        Exécute des enchères séquentielles pour assigner les tâches aux taxis.

        :param use_regret: Booléen pour activer ou non le calcul basé sur le regret.
        """
        auction_type = "Séquentielles avec Regret" if use_regret else "Séquentielles"
        print(f"\n--- Début des Enchères {auction_type} ---")

        unassigned_tasks = [task for task in self.tasks if not task.is_assigned]

        if not unassigned_tasks:
            print("Aucune tâche à assigner.")
            return

        for task in unassigned_tasks:
            best_taxi = None
            best_value = float('inf')

            for taxi in self.taxis:
                current_cost = self.calculate_marginal_cost(taxi, task)

                if use_regret:
                    future_regret = self.estimate_future_regret(taxi, task)
                    total_value = current_cost + future_regret
                    print(f"Taxi {taxi.taxi_id} - Coût actuel: {current_cost:.2f}, Regret futur: {future_regret:.2f}, Total: {total_value:.2f}")
                else:
                    total_value = current_cost
                    print(f"Taxi {taxi.taxi_id} propose un coût de {current_cost:.2f} pour la tâche {task.task_id}.")

                if total_value < best_value:
                    best_value = total_value
                    best_taxi = taxi

            if best_taxi:
                best_taxi.assign_task(task)
                task.is_assigned = True
                result_text = f"regret total de {best_value:.2f}" if use_regret else f"coût de {best_value:.2f}"
                print(f"Tâche {task.task_id} assignée au Taxi {best_taxi.taxi_id} avec {result_text}.")
            else:
                print(f"Aucune offre valide pour la tâche {task.task_id}.")

        print(f"--- Fin des Enchères {auction_type} ---\n")

    def estimate_future_regret(self, taxi, task):
        """
        Estime le regret futur pour un taxi si la tâche est assignée.

        :param taxi: Instance du taxi concerné.
        :param task: Instance de la tâche à évaluer.
        :return: Regret futur estimé.
        """
        remaining_tasks = [t for t in self.tasks if not t.is_assigned and t != task]
        regret = 0

        for future_task in remaining_tasks:
            future_cost_with_task = self.calculate_marginal_cost(taxi, future_task)
            future_cost_without_task = self.calculate_marginal_cost(taxi, future_task)
            regret += max(0, future_cost_with_task - future_cost_without_task)

        return regret


## 1.3 Lancement de la Simulation
Nous allons maintenant exécuter la simulation et observer les résultats.

In [ ]:
env1 = TaxiEnvironment(grid_size=10, num_taxis=3, task_frequency=3)
env1.run_simulation(total_steps=30, show_animation=True)

total_taches_p1 = sum(taxi.tasks_completed for taxi in env1.taxis)
cout_total_p1 = sum(taxi.execution_cost for taxi in env1.taxis)

In [ ]:
grid_sizes_p1 = [5, 10, 15]
num_taxis_list_p1 = [3]
task_frequencies_p1 = [3, 5, 7]

results_p1 = []

# Boucle à travers les différentes combinaisons de paramètres
for grid_size in grid_sizes_p1:
    for num_taxis in num_taxis_list_p1:
        for task_frequency in task_frequencies_p1:
            print(f"\nSimulation avec une grille de taille {grid_size}, {num_taxis} taxis, fréquence des tâches {task_frequency}")

            env = TaxiEnvironment(grid_size=grid_size, num_taxis=num_taxis, task_frequency=task_frequency)
            env.run_simulation(total_steps=30, show_animation=True)

            total_taches = sum(taxi.tasks_completed for taxi in env.taxis)
            cout_total = sum(taxi.execution_cost for taxi in env.taxis)

            print(f"Nombre de Tâches Complétées: {total_taches}")
            print(f"Coût Total: {cout_total}")
            print("="*83, "\n")

            results_p1.append({
                'grid_size': grid_size,
                'num_taxis': num_taxis,
                'task_frequency': task_frequency,
                'tasks_completed': total_taches,
                'total_cost': cout_total
            })

df_results_p1 = pd.DataFrame(results_p1)
moyenne_taches_completees_p1 = df_results_p1['tasks_completed'].mean()
moyenne_couts_p1 = df_results_p1['total_cost'].mean()


# 🤖 Partie 2 : Optimisation de Contraintes Distribuées (DCOP)

Dans cette partie, nous allons modéliser l'allocation de tâches en utilisant des DCOP avec la bibliothèque **Pydcop**.

## Génération du fichier YAML et exécution du solveur PyDCOP

Cette cellule définit les fonctions nécessaires pour :
1. Générer dynamiquement un fichier YAML représentant le problème DCOP.
2. Exécuter le solveur PyDCOP avec les paramètres spécifiés.
3. Simuler le système d'assignation des tâches aux taxis.

In [ ]:
# Fonction pour générer le fichier YAML pour le DCOP
def generate_dcop_yaml(environment, tasks, taxis, filename):
    """
    Génère dynamiquement un fichier YAML pour le problème DCOP.

    :param tasks: Liste des tâches à assigner.
    :param taxis: Liste des taxis disponibles.
    :param filename: Nom du fichier YAML à générer.
    """
    dcop_data = {
        'name': 'taxi_task_allocation',
        'objective': 'min',
        'domains': {},
        'variables': {},
        'constraints': {},
        'agents': {f'taxi_{taxi.taxi_id}': {'capacity': 10} for taxi in taxis}
    }

    # Création des domaines de valeurs possibles pour chaque taxi
    task_ids = [f'task_{task.task_id}' for task in tasks]
    possible_task_sets = []
    for i in range(1, len(task_ids)):
        possible_task_sets.extend(['-'.join(comb) for comb in combinations(task_ids, i)])

    for taxi in taxis:
        dcop_data['domains'][f'domain_taxi_{taxi.taxi_id}'] = {'values': possible_task_sets}
        dcop_data['variables'][f'taxi_{taxi.taxi_id}'] = {'domain': f'domain_taxi_{taxi.taxi_id}'}

    # Définition des contraintes individuelles de coûts pour chaque taxi
    for taxi in taxis:
        constraint_expr = []
        for task_set in dcop_data['domains'][f'domain_taxi_{taxi.taxi_id}']['values']:
            task_list = task_set.split('-')
            total_cost = sum(
                environment.calculate_marginal_cost(taxi, next(t for t in tasks if f'task_{t.task_id}' == task_id))
                for task_id in task_list
            )
            constraint_expr.append(f"({total_cost} if taxi_{taxi.taxi_id} == '{task_set}' else 0)")

        dcop_data['constraints'][f'cost_taxi_{taxi.taxi_id}'] = {
            'type': 'intention',
            'function': ' + '.join(constraint_expr),
            'variables': [f'taxi_{taxi.taxi_id}']
        }

    # Contrainte globale pour s'assurer que chaque tâche est assignée une seule fois
    task_assignment_constraints = []
    for task_id in task_ids:
        assignment_expr = ' + '.join([f"(1 if '{task_id}' in taxi_{taxi.taxi_id} else 0)" for taxi in taxis])
        task_assignment_constraints.append(f"(0 if ({assignment_expr}) == 1 else 10000)")

    dcop_data['constraints']['all_tasks_assigned_once'] = {
        'type': 'intention',
        'function': ' + '.join(task_assignment_constraints),
        'variables': [f'taxi_{taxi.taxi_id}' for taxi in taxis]
    }

    # Écriture du fichier YAML
    with open(filename, 'w') as file:
        yaml.dump(dcop_data, file, default_flow_style=False)

# Fonction pour exécuter le solveur PyDCOP et récupérer les solutions
def run_dcop_solver(filename, algorithm, distribution, stop_cycle, verbose=False):
    """
    Exécute le solveur PyDCOP sur le fichier YAML généré et récupère les solutions d'assignation.

    :param filename: Nom du fichier YAML.
    :param algorithm: Algorithme DCOP à utiliser.
    :param distribution: Type de distribution à utiliser.
    :param verbose: Booléen pour afficher des informations détaillées.
    :return: Dictionnaire des assignations.
    """
    try:
        result = subprocess.run(['pydcop', 'solve', '--algo', algorithm, '--distribution', distribution,
                                 '--algo_param', f'stop_cycle:{stop_cycle}', '--collect_on', 'value_change',
                                 '--run_metrics', f'{os.path.splitext(filename)[0]}_metrics.csv', filename],
                                stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, timeout=90)

        if verbose:
            print("\n--- Solver Output ---")
            print(result.stdout)
            if result.stderr:
                print("\n--- Solver Errors ---")

        if result.returncode != 0:
            print(f"Erreur : {result.stderr}")
            return None

        solver_output = json.loads(result.stdout)
        assignment = solver_output.get('assignment', {})

        if verbose:
            print("\n--- Parsed Assignment ---")
            print(assignment)

        return assignment

    except subprocess.TimeoutExpired:
        print("\n--- Solver Timeout ---")
        return None

# Boucle de simulation pour mettre à jour la disponibilité des taxis et l'assignation des tâches
def simulate_system(environment, total_steps=10, num_tasks=None, algorithm='mgm', distribution=None, stop_cycle=None,
                     filename='taxi_dcop.yml', verbose=False, show_yml=False, show_animation=False, save_to_csv=False):
    """
    Exécute la simulation de l'environnement avec l'assignation dynamique des tâches.

    :param environment: Instance de l'environnement de simulation des taxis.
    :param total_steps: Nombre total de pas de temps pour la simulation.
    :param num_tasks: Nombre de tâches à générer à chaque étape (par défaut None pour un comportement dynamique).
    :param algorithm: Algorithme DCOP à utiliser (par exemple 'mgm', 'dsa', 'maxsum').
    :param distribution: Méthode de distribution des tâches (par exemple 'oneagent', 'adhoc', 'ilp').
    :param stop_cycle: Nombre maximal de cycles pour l'algorithme (par défaut None pour la valeur par défaut de l'algorithme).
    :param filename: Nom du fichier YAML pour le solveur DCOP.
    :param verbose: Booléen pour afficher des détails supplémentaires pendant la simulation.
    :param show_yml: Booléen pour afficher le contenu du fichier YAML généré.
    :param show_animation: Booléen pour afficher l'animation de la simulation.
    :param save_to_csv: Booléen pour sauvegarder les résultats dans des fichiers CSV.
    """
    Task.task_counter = 0

    for time_step in range(total_steps):
        #if verbose:
        print(f"\nTime Step {time_step}")

        if time_step % environment.task_frequency == 0:
            num_taxis = len(environment.taxis)
            num_new_tasks = np.random.randint(num_taxis, num_taxis + 3) if num_tasks is None else num_tasks
            new_tasks = [environment.generate_task() for _ in range(num_new_tasks)]
            environment.tasks.extend(new_tasks)

        unassigned_tasks = [task for task in environment.tasks if not task.is_assigned]

        if not unassigned_tasks:
            if verbose:
                print("No tasks to assign. Skipping solver...")
            for taxi in environment.taxis:
                taxi.update_position()
                if verbose:
                    print(f"taxi_{taxi.taxi_id} - Position: {taxi.position}, Tâches: {taxi.schedule}, Phase: {taxi.task_phase}")
            if show_animation:
                environment.history.append(environment.capture_state())
            else:
                environment.display_grid()
            continue

        generate_dcop_yaml(environment, unassigned_tasks, taxis=environment.taxis, filename=filename)

        if show_yml:
            with open(filename, 'r') as file:
                print("\n--- DCOP YAML Content ---")
                print(file.read())

        if distribution is None:
            distribution = 'oneagent'
            if environment.grid_size <= 10 and environment.num_taxis <= 5 and environment.task_frequency <= 5:
                distribution = 'adhoc'

        if stop_cycle is None:
            stop_cycle = 1000
            if grid_size <= 5 and num_taxis <= 5:
                stop_cycle = 300
            elif grid_size <= 10 and num_taxis <= 7:
                stop_cycle = 500

        solution = run_dcop_solver(filename=filename, algorithm=algorithm, distribution=distribution, stop_cycle=stop_cycle, verbose=verbose)

        if solution:
            assigned_tasks = set()
            for taxi_var, task_sequence in solution.items():
                taxi = next((taxi for taxi in environment.taxis if f'taxi_{taxi.taxi_id}' == taxi_var), None)

                if not taxi:
                    continue

                task_ids = task_sequence.split('-') if task_sequence else []

                for task_id in task_ids:
                    if task_id in assigned_tasks:
                        continue

                    task = next((task for task in environment.tasks if f'task_{task.task_id}' == task_id), None)
                    if task and not task.is_assigned:
                        taxi.assign_task(task)
                        task.is_assigned = True
                        assigned_tasks.add(task_id)
                        if verbose:
                            print(f"Tâche {task.task_id} assignée à {taxi.taxi_id}")
                    elif task:
                        if verbose:
                            print(f"Tâche {task.task_id} est déjà assignée.")
                    else:
                        if verbose:
                            print(f"Erreur: Tâche {task_id} introuvable.")

        for taxi in environment.taxis:
            taxi.update_position()
            if verbose:
                print(f"taxi_{taxi.taxi_id} - Position: {taxi.position}, Tâches: {taxi.schedule}, Phase: {taxi.task_phase}")

        if show_animation:
            environment.history.append(environment.capture_state())
        else:
            environment.display_grid()

    if show_animation:
        display(environment.animate_simulation())

    environment.log_performance(save_to_csv)


## Fonction pour lancer une simulation avec mesure des performances

Cette cellule définit la fonction `run_experiment`, qui exécute une simulation en utilisant un algorithme DCOP choisi. Elle mesure :
- Le temps de résolution.
- Le coût total des solutions.

Elle renvoie ces résultats sous forme de dictionnaire pour une analyse ultérieure.

In [ ]:
def run_experiment(env, total_steps=20, num_tasks=None, algorithm='dsa', distribution=None, stop_cycle=None, verbose=False, show_yml=False, show_animation=True):
    """
    Exécute une simulation pour un algorithme donné et mesure le temps de résolution,
    la qualité des solutions, ainsi que le nombre de tâches complétées.

    :param env: Instance de l'environnement de simulation.
    :param total_steps: Nombre total d'étapes de la simulation.
    :param num_tasks: Nombre de tâches à générer à chaque étape.
    :param algorithm: Algorithme PyDCOP à utiliser.
    :param distribution: Mode de distribution pour PyDCOP.
    :param stop_cycle: Nombre maximal de cycles pour l'algorithme.
    :param verbose: Affiche les détails pendant la simulation.
    :param show_yml: Affiche le fichier YAML généré.
    :param show_animation: Affiche l'animation de la simulation.

    :return: Dictionnaire contenant :
        - 'total_time': Temps total d'exécution (en secondes).
        - 'total_cost': Coût total des solutions générées.
        - 'tasks_completed': Nombre total de tâches complétées.
        - 'algorithm': Algorithme utilisé.
    """
    start_time = time.time()
    simulate_system(env, total_steps=total_steps, num_tasks=num_tasks, algorithm=algorithm, distribution=distribution, stop_cycle=stop_cycle, verbose=verbose, show_yml=show_yml, show_animation=show_animation)
    end_time = time.time()

    total_time = end_time - start_time
    total_cost = sum(taxi.execution_cost for taxi in env.taxis)
    tasks_completed = sum(taxi.tasks_completed for taxi in env.taxis)

    return {
        'algorithm': algorithm,
        'total_time': total_time,
        'total_cost': total_cost,
        'tasks_completed': tasks_completed
    }

## Exécution d'une simulation avec visualisation des résultats

Ici, nous exécutons une simulation avec l'algorithme **DSA**. L'animation est activée pour observer le comportement des taxis et l'évolution de l'assignation des tâches.

In [ ]:
env2_a = TaxiEnvironment(grid_size=10, num_taxis=3, task_frequency=3)
run_experiment(env2_a, total_steps=30, algorithm='dsa', stop_cycle=1000, verbose=True, show_animation=True)


## Exécution d'une simulation sans animation

Cette cellule lance la même simulation que précédemment, mais sans afficher l'animation.

In [ ]:
env2_b = TaxiEnvironment(grid_size=10, num_taxis=3, task_frequency=3)
run_experiment(env2_b, total_steps=30, algorithm='dsa', stop_cycle=1000, verbose=True, show_animation=False)


## Comparaison des performances des algorithmes DSA et MGM

Cette boucle itère sur différentes tailles de grille, fréquences de tâches et algorithmes (DSA et MGM) pour comparer :
- Le temps de résolution.
- Le coût total des solutions.

Les résultats sont stockés pour une analyse comparative et la génération de graphiques.

In [ ]:
grid_sizes = [5, 10, 15]
num_taxis_list = [3] # , 5, 10
task_frequencies = [3, 5, 7]
algorithms = ['dsa', 'mgm'] #, 'maxsum', 'dpop']

results = []

for grid_size in grid_sizes:
    for num_taxis in num_taxis_list:
        for task_frequency in task_frequencies:
            for algorithm in algorithms:
                num_tasks = num_taxis if algorithm == 'maxsum' else None

                print(f"\nExécution de {algorithm} avec une grille de taille {grid_size}, {num_taxis} taxis, fréquence des tâches {task_frequency}")

                env = TaxiEnvironment(grid_size=grid_size, num_taxis=num_taxis, num_tasks=num_tasks, task_frequency=task_frequency)

                result = run_experiment(env, total_steps=30, algorithm=algorithm, distribution= None if algorithm == 'dsa' else "oneagent")
                print(f"Temps de Résolution: {result['total_time']:.2f} secondes")
                print(f"Coût Total: {result['total_cost']}")
                print(f"Tâches Complétées: {result['tasks_completed']}")
                print("="*82, "\n")
                result.update({'grid_size': grid_size, 'num_taxis': num_taxis, 'task_frequency': task_frequency})

                results.append(result)


## Visualisation des Performances des Algorithmes DCOP

Cette cellule génère plusieurs graphiques pour analyser et comparer les performances des algorithmes **DSA** et **MGM**. Les visualisations incluent :
1. Le temps de résolution moyen pour chaque algorithme.
2. La qualité des solutions en fonction du coût total généré par chaque algorithme.
3. L'impact de la taille de la grille sur le temps de résolution.
4. L'impact de la fréquence des tâches sur le temps de résolution.

Ces graphiques permettent d'identifier les forces et les faiblesses de chaque algorithme dans différents scénarios.

In [ ]:
df_results = pd.DataFrame(results)

# ==========================
# Comparaison du Temps de Résolution entre Algorithmes DCOP
# ==========================
resolution_times = df_results.groupby('algorithm')['total_time'].mean().to_dict()

plt.figure(figsize=(8, 6))
plt.bar(list(resolution_times.keys()), list(resolution_times.values()), color='orange')
plt.xlabel('Algorithmes DCOP')
plt.ylabel('Temps de Résolution (s)')
plt.title('Comparaison du Temps de Résolution entre Algorithmes DCOP')
plt.show()

# ==========================
# Comparaison de la Qualité des Solutions (Coût Total)
# ==========================
solution_costs = df_results.groupby('algorithm')['total_cost'].mean().to_dict()

plt.figure(figsize=(8, 6))
plt.bar(list(solution_costs.keys()), list(solution_costs.values()), color='purple')
plt.xlabel('Algorithmes DCOP')
plt.ylabel('Coût Total')
plt.title('Qualité des Solutions par Algorithme DCOP')
plt.show()

# ==========================
# Impact de la Taille de la Grille sur le Temps de Résolution
# ==========================
plt.figure(figsize=(10, 6))
for algorithm in df_results['algorithm'].unique():
    data = df_results[df_results['algorithm'] == algorithm]
    avg_resolution_time = data.groupby('grid_size')['total_time'].mean()
    plt.plot(avg_resolution_time.index, avg_resolution_time.values, label=algorithm)

plt.xlabel('Taille de la Grille')
plt.ylabel('Temps de Résolution Moyen (s)')
plt.title('Impact de la Taille de la Grille sur le Temps de Résolution')
plt.legend()
plt.grid(True)
plt.show()

# ==========================
# Impact du Nombre de Taxis sur la Qualité des Solutions
# ==========================
# plt.figure(figsize=(10, 6))
# for algorithm in df_results['algorithm'].unique():
#     data = df_results[df_results['algorithm'] == algorithm]
#     avg_total_cost = data.groupby('num_taxis')['total_cost'].mean()
#     plt.plot(avg_total_cost.index, avg_total_cost.values, label=algorithm)
#
# plt.xlabel('Nombre de Taxis')
# plt.ylabel('Coût Total Moyen')
# plt.title('Impact du Nombre de Taxis sur la Qualité des Solutions')
# plt.legend()
# plt.grid(True)
# plt.show()

# ==========================
# Impact de la Fréquence des Tâches sur le Temps de Résolution
# ==========================
plt.figure(figsize=(10, 6))
for algorithm in df_results['algorithm'].unique():
    data = df_results[df_results['algorithm'] == algorithm]
    avg_resolution_time = data.groupby('task_frequency')['total_time'].mean()
    plt.plot(avg_resolution_time.index, avg_resolution_time.values, label=algorithm)

plt.xlabel('Fréquence des Tâches')
plt.ylabel('Temps de Résolution Moyen (s)')
plt.title('Impact de la Fréquence des Tâches sur le Temps de Résolution')
plt.legend()
plt.grid(True)
plt.show()

# ==========================
# Comparaison du Nombre de Tâches Complétées entre Algorithmes DCOP
# ==========================
tasks_completed_avg = df_results.groupby('algorithm')['tasks_completed'].mean().to_dict()

plt.figure(figsize=(8, 6))
plt.bar(list(tasks_completed_avg.keys()), list(tasks_completed_avg.values()), color='green')
plt.xlabel('Algorithmes DCOP')
plt.ylabel('Nombre de Tâches Complétées')
plt.title('Comparaison du Nombre de Tâches Complétées entre Algorithmes DCOP')
plt.show()

# ==========================
# Analyse des Paramètres Similaires à la Partie 1
# ==========================
# Filtrage des résultats pour la grille de taille 10, 3 taxis et une fréquence de 3
filtered_results_p2 = df_results[(df_results['grid_size'] == 10) &
                                 (df_results['num_taxis'] == 3) &
                                 (df_results['task_frequency'] == 3)]

# Moyenne des coûts et du temps pour les algorithmes (Paramètres similaires à la Partie 1)
moyenne_couts_p2 = filtered_results_p2.groupby('algorithm')['total_cost'].mean().to_dict()
moyenne_temps_p2 = filtered_results_p2.groupby('algorithm')['total_time'].mean().to_dict()
moyenne_taches_completees_p2 = filtered_results_p2.groupby('algorithm')['tasks_completed'].mean().to_dict()


# 💬 Partie 3 : Protocoles de Négociation Multi-Agents

Nous implémenterons des enchères séquentielles et parallèles pour évaluer leurs performances dans l'assignation des tâches.

In [ ]:
grid_sizes = [5, 10, 15]
num_taxis_list = [3]
task_frequencies = [3, 5, 7]
auction_types = [AuctionType.PSI, AuctionType.SSI, AuctionType.SSI_REGRET]

results_auctions = []

for grid_size in grid_sizes:
    for num_taxis in num_taxis_list:
        for task_frequency in task_frequencies:
            for auction_type in auction_types:
                print(f"\nExécution de {auction_type.value} avec une grille de taille {grid_size}, {num_taxis} taxis, fréquence des tâches {task_frequency}")

                env = TaxiEnvironment(grid_size=grid_size, num_taxis=num_taxis, task_frequency=task_frequency, auction_type=auction_type)
                env.run_simulation(total_steps=30, show_animation=True)

                total_taches_completees = sum(taxi.tasks_completed for taxi in env.taxis)
                cout_total = sum(taxi.execution_cost for taxi in env.taxis)

                results_auctions.append({
                    'auction_type': auction_type.value,
                    'grid_size': grid_size,
                    'num_taxis': num_taxis,
                    'task_frequency': task_frequency,
                    'tasks_completed': total_taches_completees,
                    'total_cost': cout_total
                })

df_auctions = pd.DataFrame(results_auctions)

# Calcul des moyennes pour les analyses
moyenne_taches_completees_p3 = df_auctions.groupby('auction_type')['tasks_completed'].mean().to_dict()
moyenne_couts_enchere_p3 = df_auctions.groupby('auction_type')['total_cost'].mean().to_dict()

print("="*82)
env.plot_completed_tasks_by_auction(moyenne_taches_completees_p3)
env.plot_auction_cost_comparison(moyenne_couts_enchere_p3)

# 📊 Comparaison Globale des Performances entre les Différentes Approches

Dans cette section, nous comparons les performances des différentes approches étudiées dans les trois parties du projet.
Les métriques analysées incluent le **coût total moyen** et le **nombre de tâches complétées**.

- **Partie 1** représente la simulation de base sans algorithme d'optimisation complexe.
- **Partie 2** analyse l'impact des algorithmes DCOP (**DSA** et **MGM**) sur la qualité des solutions.
- **Partie 3** explore les performances des différentes stratégies d'enchères (**PSI**, **SSI**, et **SSI avec regret**).

Ces visualisations nous permettent d'identifier les approches les plus efficaces en termes d'optimisation des coûts et de volume de tâches accomplies.


In [ ]:
labels = ['Partie 1', 'Partie 2 - DSA', 'Partie 2 - MGM', 'Partie 3 - PSI', 'Partie 3 - SSI', 'Partie 3 - SSI avec regret']

couts = [
    moyenne_couts_p1,
    moyenne_couts_p2.get('dsa', 0),
    moyenne_couts_p2.get('mgm', 0),
    moyenne_couts_enchere_p3.get(AuctionType.PSI.value, 0),
    moyenne_couts_enchere_p3.get(AuctionType.SSI.value, 0),
    moyenne_couts_enchere_p3.get(AuctionType.SSI_REGRET.value, 0)
]

plt.figure(figsize=(10, 6))
plt.bar(labels, couts, color=['blue', 'red', 'red', 'green', 'green', 'green'])
plt.xlabel('Méthodes')
plt.ylabel('Coût Total Moyen')
plt.title('Comparaison des Coûts Totaux entre les Parties')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.show()


taches_completes = [
    moyenne_taches_completees_p1,
    moyenne_taches_completees_p2.get('dsa', 0),
    moyenne_taches_completees_p2.get('dsa', 0),
    moyenne_taches_completees_p3.get(AuctionType.PSI.value, 0),
    moyenne_taches_completees_p3.get(AuctionType.SSI.value, 0),
    moyenne_taches_completees_p3.get(AuctionType.SSI_REGRET.value, 0)
]

plt.figure(figsize=(10, 6))
plt.bar(labels, taches_completes, color=['blue', 'red', 'red', 'green', 'green', 'green'])
plt.xlabel('Méthodes')
plt.ylabel('Nombre de Tâches Complétées')
plt.title('Comparaison des Tâches Complétées entre les Parties')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.show()